In [97]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN
from sklearn.metrics import r2_score

In [3]:
df=pd.read_excel('/content/drive/MyDrive/Agritech/Agmarknet_Price_And_Arrival_Report_MP.xlsx', header=1)
df.head()

,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Reported Date
0,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,71.25,2880,4665,2230,2023-12-30
1,Madhya Pradesh,Ashoknagar,Ashoknagar,Lokwan,Cereals,210.83,2891,2891,2288,2023-12-29
2,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,NaN,3085,4701,2288,2023-12-29
3,Madhya Pradesh,Ashoknagar,Ashoknagar,Other,Cereals,NaN,3150,3150,2288,2023-12-29
4,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,165.80,2776,4666,2500,2023-12-28


In [14]:
df['Reported Date']=pd.to_datetime(df['Reported Date'])
df['Year']=df['Reported Date'].dt.year
df.head()

,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Reported Date,Year
0,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,71.25,2880,4665,2230,2023-12-30,2023
1,Madhya Pradesh,Ashoknagar,Ashoknagar,Lokwan,Cereals,210.83,2891,2891,2288,2023-12-29,2023
2,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,NaN,3085,4701,2288,2023-12-29,2023
3,Madhya Pradesh,Ashoknagar,Ashoknagar,Other,Cereals,NaN,3150,3150,2288,2023-12-29,2023
4,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,165.80,2776,4666,2500,2023-12-28,2023


In [19]:
df_train=df[df['Year']!=2023]
df_test=df[df['Year']==2023]
df_test['Year'].unique()

array([2023], dtype=int32)

In [126]:
class Crop_Price_Prediction:
  def __init__(self, df_train, df_test):
    self.df_train=df_train
    self.df_test=df_test
    # self.train=self.df[self.df['']]

  def prepare_train_data(self, n_features):

    timeseries=self.df_train['Modal Price (Rs./Quintal)'].values
    X,y=[], []
    for i in range(len(timeseries)):
      end_ix = i + n_features

      if end_ix > len(timeseries)-1:
        break

      seq_x, seq_y= timeseries[i:end_ix], timeseries[end_ix]
      X.append(seq_x)
      y.append(seq_y)

    return np.array(X), np.array(y)

  def prepare_test_data(self, n_features):

    timeseries=self.df_test['Modal Price (Rs./Quintal)'].values
    X,y=[], []
    for i in range(len(timeseries)):
      end_ix = i + n_features

      if end_ix > len(timeseries)-1:
        break

      seq_x, seq_y= timeseries[i:end_ix], timeseries[end_ix]
      X.append(seq_x)
      y.append(seq_y)

    return np.array(X), np.array(y)


  # def date_conversion(self, col):

  #     self.df[col]=pd.to_datetime(self.df[col])
  #     self.df['Year']=self.df[col].dt.year

  def head(self):
    return self.df.head()


  def lstm_model(self, X_train, n_steps=15, n_features=1):
    self.model = Sequential()
    self.model.add(SimpleRNN(128, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    self.model.add(SimpleRNN(64, activation = 'relu'))
    self.model.add(SimpleRNN(32, activation = 'relu'))
    self.model.add(SimpleRNN(50, activation = 'relu'))
    self.model.add(Dense(1))
    self.model.compile(optimizer='adam', loss='mse')

  def lstm_training(self, X_train, y_train):
    self.model.fit(X_train, y_train, epochs=50)

  def lstm_predict(self, X_test, y_test):
    y_pred=self.model.predict(X_test)
    print(r2_score(y_pred, y_test))

cp=Crop_Price_Prediction(df_train, df_test)

In [127]:
df_train['Year'].values

array([2022, 2022, 2022, ..., 2016, 2016, 2016], dtype=int32)

In [128]:
X_train, y_train = cp.prepare_train_data(n_features=15)

In [129]:
X_train.shape, y_train.shape

((8001, 15), (8001,))

In [130]:
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train.shape

(8001, 15, 1)

In [131]:
cp.lstm_model(X_train)
cp.lstm_training(X_train, y_train)

ValueError: Input 0 of layer "simple_rnn_11" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)

In [ ]:
X_test, y_test = cp.prepare_test_data(30)

In [94]:
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [95]:
X_test.shape


(711, 30, 1)

In [96]:
cp.lstm_predict(X_test, y_test)

23/23 [==============================] - 1s 8ms/step
-0.9397407128352091
